In [2]:
%pip install pandas seaborn matplotlib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# 데이터 불러오기
customers = pd.read_csv(r"C:\Users\컴퓨터\Documents\GitHub\DATA_11_BC\기초_프로젝트\my-project\data\customer_hm.csv")
transactions = pd.read_csv(r"C:\Users\컴퓨터\Documents\GitHub\DATA_11_BC\기초_프로젝트\my-project\data\transactions_hm.csv")
articles = pd.read_csv(r"C:\Users\컴퓨터\Documents\GitHub\DATA_11_BC\기초_프로젝트\my-project\data\articles_hm.csv")

In [4]:
### 결측치 및 이상치 처리
# 1. 범주형 데이터 정리 (fashion_news_frequency)
# 결측치(NaN)를 'NONE'으로 채우기
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].fillna('NONE')

# 2. 희소 카테고리 통합 (Articles)
# 상품 그룹이 50개 미만인 경우 'Others'로 변경
counts = articles['product_group_name'].value_counts()
rare_groups = counts[counts < 50].index

articles['product_group_name'] = articles['product_group_name'].replace(rare_groups, 'Others')

# 3. 결측치 처리 (detail_desc)
# 결측치를 'No Description'으로 채우기
articles['detail_desc'] = articles['detail_desc'].fillna('No Description')

# 4. 이상치 처리 (perceived_colour_master_id)
articles = articles[articles['perceived_colour_master_id'] != -1]
print(f"상품 데이터 삭제 후 크기: {articles.shape}")

#========================================================================================================

### 데이터 형 변환 및 연령대 파생 컬럼 생성
# 5. articles 데이터 변환 (거의 모든 object 컬럼 변환)
for col in articles.columns:
    if articles[col].dtype == 'object' and col != "detail_desc":  # 문자열 컬럼만 선택, detail_desc 제외
        # 고유값이 전체 데이터의 50% 미만인 경우만 변환 (효율적인 기준)
        if articles[col].nunique() < len(articles) * 0.5:
            articles[col] = articles[col].astype('category')

# 6. customers 데이터 변환 (지정해서 변환)
# customer_id는 고유값이 너무 많아 제외합니다.
cols_to_convert = ['fashion_news_frequency', 'club_member_status']
for col in cols_to_convert:
    customers[col] = customers[col].astype('category')

# 7. FN, Active 컬럼 최적화 (bool 또는 int8)
# 방법 A: True/False로 보고 싶을 때 (가장 추천)
customers['FN'] = customers['FN'].astype('bool')
customers['Active'] = customers['Active'].astype('bool')

# 8. 날짜 처리(노션 참조)
transactions["t_dat"] = pd.to_datetime(transactions["t_dat"])
transactions["year"] = transactions["t_dat"].dt.year
transactions["month"] = transactions["t_dat"].dt.month
transactions["weekday"] = transactions["t_dat"].dt.day_name()

# 9. 연령대 컬럼 및 형 변환(노션 참조)(필요 시 형 변환 제거 가능)
customers["age_group"] = (customers["age"] // 10) * 10
customers["age_group"] = customers["age_group"].astype("category")

# 10. -> Offline, 2 -> Online 으로 매핑 후 카테고리 변환
transactions['sales_channel'] = transactions['sales_channel_id'].map({1: 'Offline', 2: 'Online'}).astype('category')

# 기존 sales_channel_id 컬럼은 필요 시 삭제 가능
# transactions = transactions.drop(columns=['sales_channel_id'])

상품 데이터 삭제 후 크기: (104857, 25)


In [5]:
customers_c = customers.copy()
transactions_c = transactions.copy()
articles_c = articles.copy()

In [6]:
customers_c.to_csv('data/customers_c.csv', index=False)
transactions_c.to_csv('data/transactions_c.csv', index=False)
articles_c.to_csv('data/articles_c.csv', index=False)

In [7]:
customers_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype   
---  ------                  --------------    -----   
 0   customer_id             1048575 non-null  object  
 1   FN                      1048575 non-null  bool    
 2   Active                  1048575 non-null  bool    
 3   club_member_status      1048575 non-null  category
 4   fashion_news_frequency  1048575 non-null  category
 5   age                     1048575 non-null  int64   
 6   age_group               1048575 non-null  category
dtypes: bool(2), category(3), int64(1), object(1)
memory usage: 21.0+ MB


In [16]:
customers_c["club_member_status"].value_counts()

club_member_status
ACTIVE        982635
PRE-CREATE     65581
LEFT CLUB        359
Name: count, dtype: int64

In [8]:
transactions_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 9 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   t_dat             1048575 non-null  datetime64[ns]
 1   customer_id       1048575 non-null  object        
 2   article_id        1048575 non-null  int64         
 3   price             1048575 non-null  float64       
 4   sales_channel_id  1048575 non-null  int64         
 5   year              1048575 non-null  int32         
 6   month             1048575 non-null  int32         
 7   weekday           1048575 non-null  object        
 8   sales_channel     1048575 non-null  category      
dtypes: category(1), datetime64[ns](1), float64(1), int32(2), int64(2), object(2)
memory usage: 57.0+ MB


In [9]:
articles_c.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104857 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype   
---  ------                        --------------   -----   
 0   article_id                    104857 non-null  int64   
 1   product_code                  104857 non-null  int64   
 2   prod_name                     104857 non-null  category
 3   product_type_no               104857 non-null  int64   
 4   product_type_name             104857 non-null  category
 5   product_group_name            104857 non-null  category
 6   graphical_appearance_no       104857 non-null  int64   
 7   graphical_appearance_name     104857 non-null  category
 8   colour_group_code             104857 non-null  int64   
 9   colour_group_name             104857 non-null  category
 10  perceived_colour_value_id     104857 non-null  int64   
 11  perceived_colour_value_name   104857 non-null  category
 12  perceived_colour_master_id    10485